In [0]:
import pandas as pd
import os
import tqdm

path = "/content/drive/My Drive/APS360 Project/v6-dataset/"

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Extract image ID from annotation file
def extract_img_id():
    extracted_class = pd.read_csv(os.path.join(path,'class-descriptions.csv'))
   
    data = ['test', 'validation', 'train']

    for i in data:
        extracted = {}
        extracted['ImageID'], extracted['Class_ID'], extracted['Class_Description'] = [],[],[]
        extracted['XMin'], extracted['XMax'], extracted['YMin'], extracted['YMax'] = [],[],[],[]
        file_path = os.path.join(path, i+'-annotations-bbox.csv')
        image_id_csv = pd.read_csv(file_path,
                                   usecols=['ImageID','LabelName','XMin','XMax','YMin','YMax'])
        bar = tqdm.tqdm(total=len(image_id_csv.ImageID),desc='EXTRACTING',position=0)

        for j in range(len(image_id_csv.ImageID)):
            bar.update(1)
            if image_id_csv.LabelName[j] in extracted_class['Class_ID'].values:
                idx_in_data = extracted_class.Class_ID[extracted_class.Class_ID==image_id_csv.LabelName[j]].index.tolist()[0]
                extracted['ImageID'].append(image_id_csv.ImageID[j])
                extracted['Class_ID'].append(extracted_class.Class_ID[idx_in_data])
                extracted['Class_Description'].append(extracted_class.Class_Description[idx_in_data])
                extracted['XMin'].append(image_id_csv.XMin[j])
                extracted['XMax'].append(image_id_csv.XMax[j])
                extracted['YMin'].append(image_id_csv.YMin[j])
                extracted['YMax'].append(image_id_csv.YMax[j])
        
        # Export file
        export_csv = pd.DataFrame(extracted).to_csv(os.path.join(path, i+'.csv'),index=None,header=True)

extract_img_id()

In [0]:
# Extracted all image URLs based on image ID
def id_to_url():
    extracted_class = pd.read_csv(os.path.join(path,'class-descriptions.csv'))
   
    data = ['test', 'train', 'validation']

    for i in data:
        class_id_data = pd.read_csv(os.path.join(path,i+'.csv'))
        extracted_image_url = {}

        extracted_image_url['ImageID'], extracted_image_url['Image_title'], extracted_image_url['Class_ID'], extracted_image_url['Class_Description'] = [],[],[],[]
        extracted_image_url['XMin'], extracted_image_url['XMax'], extracted_image_url['YMin'], extracted_image_url['YMax'] = [],[],[],[]
        extracted_image_url['Image_url'] = []
        file_path = os.path.join(path, i+'-annotations-bbox.csv')
        image_url_data = pd.read_csv(os.path.join(path, i+'-images-with-rotation.csv'), usecols=['ImageID','Title','Thumbnail300KURL'])
        bar = tqdm.tqdm(total=len(image_url_data.ImageID),desc='EXTRACTING',position=0)
        for j in range(len(image_url_data.ImageID)):
            bar.update(1)
            if (image_url_data.ImageID[j] in class_id_data['ImageID'].values):
                idx_in_data = class_id_data.ImageID[class_id_data.ImageID==image_url_data.ImageID[j]].index.tolist()[0]
                extracted_image_url['ImageID'].append(image_url_data.ImageID[j])
                extracted_image_url['Image_title'].append(image_url_data.Title[j])
                extracted_image_url['Class_ID'].append(class_id_data.Class_ID[idx_in_data])
                extracted_image_url['Class_Description'].append(class_id_data.Class_Description[idx_in_data])
                extracted_image_url['Image_url'].append(image_url_data.Thumbnail300KURL[j])
                extracted_image_url['XMin'].append(class_id_data.XMin[idx_in_data])
                extracted_image_url['XMax'].append(class_id_data.XMax[idx_in_data])
                extracted_image_url['YMin'].append(class_id_data.YMin[idx_in_data])
                extracted_image_url['YMax'].append(class_id_data.YMax[idx_in_data])
                
        # Export file
        export_csv = pd.DataFrame(extracted_image_url).to_csv(os.path.join(path, i+'-id.csv'),index=None,header=True)

id_to_url()

In [0]:
# Download the images
import urllib

def download():
    img_url_data = pd.read_csv(os.path.join(path,'train-id.csv'))
    bar = tqdm.tqdm(total=len(img_url_data.Image_url),desc='Downloading',position=0)
    for j in range(len(img_url_data.Image_url)):
        bar.update(1)
        img_name = img_url_data.Class_Description[j] + '-' + img_url_data.ImageID[j] + '.png'
        file_name = os.path.join(path,'train',img_name)
        url = img_url_data.Image_url[j]
        try:
            print('Downloading from ' + url)
            urllib.request.urlretrieve(url,file_name)
            print('Success\n\n')
        except:
            print('Failed\n\n')

download()

In [0]:
def cleanTrainAnnotationFile():

    # Get a list of all the files in a directory (allImages)
    # Within the file, check if the csv entry is in the file directory
        # If it is, add to dict: | path to image | class description | bbox |
    # After going through all the images, store dict into a csv

    allImages1 = list()
    file_path = os.path.join(path, 'train-1')
    for f in os.listdir(file_path):
        allImages1.append(f)

    allImages2 = list()
    file_path = os.path.join(path, 'train-2')
    for f in os.listdir(file_path):
        allImages2.append(f)

    allImages3 = list()
    file_path = os.path.join(path, 'train-3')
    for f in os.listdir(file_path):
        allImages3.append(f)

    print(len(allImages1))
    print(len(allImages2))
    print(len(allImages3))
    print(len(allImages1) + len(allImages2) + len(allImages3))

    # Get the path to the CSV file
    headers = pd.read_csv(os.path.join(path, 'train-id.csv'))

    # This is the dictionary we will use
    extracted = {}

    extracted['filePath'], extracted['Class_Description'] = [],[],
    extracted['XMin'], extracted['XMax'], extracted['YMin'], extracted['YMax'] = [],[],[],[]

    for j in range(len(headers.ImageID)):

        # Get the file name
        fileName = headers.Class_Description[j]+'-'+headers.ImageID[j]+'.png'

        if fileName in allImages1:
            extracted['filePath'].append(os.path.join(path, 'train-1', fileName))
            extracted['Class_Description'].append(headers.Class_Description[j])
            extracted['XMin'].append(headers.XMin[j])
            extracted['XMax'].append(headers.XMax[j])
            extracted['YMin'].append(headers.YMin[j])
            extracted['YMax'].append(headers.YMax[j])
            
        elif fileName in allImages2:
            extracted['filePath'].append(os.path.join(path, 'train-2', fileName))
            extracted['Class_Description'].append(headers.Class_Description[j])
            extracted['XMin'].append(headers.XMin[j])
            extracted['XMax'].append(headers.XMax[j])
            extracted['YMin'].append(headers.YMin[j])
            extracted['YMax'].append(headers.YMax[j])

        elif fileName in allImages3:
            extracted['filePath'].append(os.path.join(path, 'train-3', fileName))
            extracted['Class_Description'].append(headers.Class_Description[j])
            extracted['XMin'].append(headers.XMin[j])
            extracted['XMax'].append(headers.XMax[j])
            extracted['YMin'].append(headers.YMin[j])
            extracted['YMax'].append(headers.YMax[j])

    # Export file
    export_csv = pd.DataFrame(extracted).to_csv(os.path.join(path, 'clean-train-id.csv'), index=None, header=True)
    print(len(extracted['filePath']))
    print(len(extracted['Class_Description']))
    print(len(extracted['XMin']))
    print(len(extracted['XMax']))
    print(len(extracted['YMin']))
    print(len(extracted['YMax']))

cleanTrainAnnotationFile()

17159
9904
6019
33082
33082
33082
33082
33082
33082
33082


In [0]:
def cleanValAnnotationFile():

    # Get a list of all the files in a directory (allImages)
    # Within the file, check if the csv entry is in the file directory
        # If it is, add to dict: | path to image | class description | bbox |
    # After going through all the images, store dict into a csv

    allImages1 = list()
    file_path = os.path.join(path, 'validation')
    for f in os.listdir(file_path):
        allImages1.append(f)

    print(len(allImages1))

    # Get the path to the CSV file
    headers = pd.read_csv(os.path.join(path, 'validation-id.csv'))

    # This is the dictionary we will use
    extracted = {}

    extracted['filePath'], extracted['Class_Description'] = [],[],
    extracted['XMin'], extracted['XMax'], extracted['YMin'], extracted['YMax'] = [],[],[],[]

    for j in range(len(headers.ImageID)):

        # Get the file name
        fileName = headers.Class_Description[j]+'-'+headers.ImageID[j]+'.png'

        if fileName in allImages1:
            extracted['filePath'].append(os.path.join(path, 'validation', fileName))
            extracted['Class_Description'].append(headers.Class_Description[j])
            extracted['XMin'].append(headers.XMin[j])
            extracted['XMax'].append(headers.XMax[j])
            extracted['YMin'].append(headers.YMin[j])
            extracted['YMax'].append(headers.YMax[j])

    # Export file
    export_csv = pd.DataFrame(extracted).to_csv(os.path.join(path, 'clean-validation-id.csv'), index=None, header=True)
    print("\n")
    print(len(extracted['filePath']))
    print(len(extracted['Class_Description']))
    print(len(extracted['XMin']))
    print(len(extracted['XMax']))
    print(len(extracted['YMin']))
    print(len(extracted['YMax']))

cleanValAnnotationFile()

2303


2303
2303
2303
2303
2303
2303


In [0]:
def cleanTestAnnotationFile():

    # Get a list of all the files in a directory (allImages)
    # Within the file, check if the csv entry is in the file directory
        # If it is, add to dict: | path to image | class description | bbox |
    # After going through all the images, store dict into a csv

    allImages1 = list()
    file_path = os.path.join(path, 'test')
    for f in os.listdir(file_path):
        allImages1.append(f)

    print(len(allImages1))

    # Get the path to the CSV file
    headers = pd.read_csv(os.path.join(path, 'test-id.csv'))

    # This is the dictionary we will use
    extracted = {}

    extracted['filePath'], extracted['Class_Description'] = [],[],
    extracted['XMin'], extracted['XMax'], extracted['YMin'], extracted['YMax'] = [],[],[],[]

    for j in range(len(headers.ImageID)):

        # Get the file name
        fileName = headers.Class_Description[j]+'-'+headers.ImageID[j]+'.png'

        if fileName in allImages1:
            extracted['filePath'].append(os.path.join(path, 'test', fileName))
            extracted['Class_Description'].append(headers.Class_Description[j])
            extracted['XMin'].append(headers.XMin[j])
            extracted['XMax'].append(headers.XMax[j])
            extracted['YMin'].append(headers.YMin[j])
            extracted['YMax'].append(headers.YMax[j])

    # Export file
    export_csv = pd.DataFrame(extracted).to_csv(os.path.join(path, 'clean-test-id.csv'), index=None, header=True)
    print("\n")
    print(len(extracted['filePath']))
    print(len(extracted['Class_Description']))
    print(len(extracted['XMin']))
    print(len(extracted['XMax']))
    print(len(extracted['YMin']))
    print(len(extracted['YMax']))

cleanTestAnnotationFile()

6873


6873
6873
6873
6873
6873
6873
